# GNNを用いたリンク予測
`introduction <1_introduction>`では、GNNを用いたノード分類 (グラフ内のノードの分類を予測する) について学習した。今回は、リンク予測 (グラフ内の任意の2つのノード間のエッジの存在を予測する) にGNNを訓練する方法を学ぶ。


このチュートリアルの目標は、以下の通りである:

- GNNを用いたリンク予測モデルを構築する。
- 小規模なDGL提供のデータセットでモデルを訓練し、評価する。

(Time estimate: 28 minutes)


In [2]:
# ライブラリのインポート
import itertools
import os

os.environ["DGLBACKEND"] = "pytorch"

import dgl
import dgl.data
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F

## GNNを用いたリンク予測の概要

social recommendationや、item recommendation、知識グラフ補完のような多くの適用例は、リンク予測として定式化できる。リンク予測は、2つの特定のノード間にエッジが存在するかどうかを予測する。このチュートリアルでは、引用ネットワーク内の2つの論文間に引用関係が存在するかどうかを予測する例を示す。

このチュートリアルでは、リンク予測問題を以下のように2値分類問題として定式化する:

- グラフ内のエッジを *positive examples* として扱う。
- 存在しないエッジ (つまり、それらの間にエッジが存在しないノードペア) を *negative examples* としてサンプリングする。
- positive examples と negative examples をトレーニングセットとテストセットに分割する。
- 任意の2値分類メトリック (例: Area Under Curve (AUC)) でモデルを評価する。

<div class="alert alert-info"><h4>Note</h4><p>The practice comes from
   [SEAL](https://papers.nips.cc/paper/2018/file/53f0d7c537d99b3824f0f99d62ea2428-Paper.pdf)_,
   although the model here does not use their idea of node labeling.</p></div>

大規模スケールのレコメンダーシステムや情報検索のようないくつかのドメインでは、上位K個の予測の良いパフォーマンスを強調するような評価指標が望ましい。この場合、平均適合率 (mean average precision) などの他の評価指標を用いてもよく、このチュートリアルの範疇を超えるが、他のネガティブサンプリング手法を適用してもよい。

## グラフと特徴量の読み込み
`introduction <1_introduction>`に引き続き、ここでもまずCoraデータセットを読み込む。

In [3]:
dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


## Prepare training and testing sets

## 学習セットとテストセットの準備

このチュートリアルでは、テストセットのpositive examplesとしてエッジの10%をランダムに選択し、残りをトレーニングセットに残す。そして、それぞれのセットに同じ数のnegative examplesをサンプリングする。

In [5]:
# エッジセットをトレーニングとテストに分割する
u, v = g.edges()

eids = np.arange(g.num_edges())
eids = np.random.permutation(eids)
test_size = int(len(eids) * 0.1)
train_size = g.num_edges() - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

# すべての負例のエッジを見つけ、トレーニングとテストに分割します
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy()))) # 隣接行列を作成 -> 補足A
adj_neg = 1 - adj.todense() - np.eye(g.num_nodes()) # 隣接行列の補集合を作成 -> 補足B
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), g.num_edges()) # 負例のエッジをランダムに選択
test_neg_u, test_neg_v = (
    neg_u[neg_eids[:test_size]],
    neg_v[neg_eids[:test_size]],
)
train_neg_u, train_neg_v = (
    neg_u[neg_eids[test_size:]],
    neg_v[neg_eids[test_size:]],
)

### 補足A 隣接行列の作成

`scipy.sparse.coo_matrix`では、value, (row, col)の形式でデータを入力することで、(row, col)の位置にvalueを代入した疎行列を作成できる。今回はエッジのインデックスを(u, v)に与えるため、`scipy.sparse.coo_matrix`を使って隣接行列を作成する。
```python
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
```

### 補足B 隣接行列の反転
`adj_neg`では、隣接行列`adj`が反転された行列が作成される。具体的には、1 - `adj`で隣接行列が反転され、 `-np.eye(num_nodes)`で対角成分が0になるように調整される。
```python
adj_neg = 1 - adj.todense() - np.eye(g.num_nodes())
```



学習するときは、テストセットのエッジを元のグラフから削除する必要がある。これは``dgl.remove_edges``を使って行うことができる。


<div class="alert alert-info"><h4>Note</h4><p>
   ``dgl.remove_edges``は元のグラフからサブグラフを作成することで動作するので、コピーが作成されることにより大規模なグラフでは遅くなる可能性がある。その場合は、前処理と同様にトレーニンググラフとテストグラフをディスクに保存してもよい。</p></div>
   
   </p></div>


In [6]:
train_g = dgl.remove_edges(g, eids[:test_size]) # テストエッジを削除

## GraphSAGEモデルの定義
このチュートリアルでは、2つの[GraphSAGE](https://arxiv.org/abs/1706.02216)層からなるモデルを構築する。  
各層は、隣接情報を平均して新しいノード表現を計算する。DGLは、GraphSAGE層を簡単に作成する``dgl.nn.SAGEConv``を提供する。  

SAGEConvは、`3_messgae_passing`で既に自作しているが、今回はDGLの`SAGEConv`を使用する。

In [7]:
from dgl.nn import SAGEConv


# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, "mean")
        self.conv2 = SAGEConv(h_feats, h_feats, "mean")

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

モデルは、両端のノードの表現間のスコアを計算することで、エッジの存在確率を予測する。スコアは、両ノードの特徴量を用いた関数 (例: MLPまたは内積)で計算される。

\begin{align}\hat{y}_{u\sim v} = f(h_u, h_v)\end{align}




## ポジティブグラフ、ネガティブグラフ、および``apply_edges``

前回のチュートリアルでは、GNNを用いてノード表現を計算する方法を学習した。しかし、リンク予測では、*ノードのペア*の表現を計算する必要がある。  


DGLは、ノードのペアをエッジとして表現できるため、ノードのペアを別のグラフとして扱うことを推奨している。リンク予測では、すべてのpositive examplesをエッジとして持つ*positive graph*と、すべてのnegative examplesを持つ*negative graph*がある。*positive graph*と*negative graph*は、元のグラフと同じノードセットを持つ。これにより、複数のグラフ間でノード特徴量を簡単に渡すことができる。後で見るように、全体のグラフで計算されたノード表現を直接positive graphとnegative graphに渡して、ペアごとのスコアを計算することができる。

以下のコードは、それぞれのトレーニングセットとテストセットに対して、positive graphとnegative graphを構築する。

In [8]:
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.num_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.num_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.num_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.num_nodes())

ノードのペアをグラフとして扱う利点は、``DGLGraph.apply_edges``メソッドを使用できることである。  
このメソッドは、便利に、インシデントノードの特徴量と元のエッジの特徴量 (適用可能な場合) に基づいて新しいエッジ特徴量を計算する。


DGLは、元のノード/エッジ特徴量に基づいて新しいエッジ特徴量を計算するための最適化された組み込み関数を提供している。  
例えば、``dgl.function.u_dot_v``は、各エッジのインシデントノードの表現の内積を計算する。

In [11]:
import dgl.function as fn

# 内積を計算するためのカスタムエッジ予測器
class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h  # ノード特徴量を追加。hはノードの特徴量を表すテンソル
            # 'score' という新しいエッジ特徴量を計算する。これは、ソースノードの特徴量 'h' とターゲットノードの特徴量 'h' の内積によって計算される。
            g.apply_edges(fn.u_dot_v("h", "h", "score"))
            # u_dot_v は各エッジに対して1要素のベクトルを返すため、それをsqueezeする必要がある
            return g.edata["score"][:, 0]

自分で関数を書くこともできる。
例えば、次のモジュールは、インシデントノードの特徴量を連結してMLPに渡し、各エッジにスカラースコアを生成する。

In [12]:
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.

        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.

        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        h = torch.cat([edges.src["h"], edges.dst["h"]], 1) # ソースノードとターゲットノードの特徴量を連結
        return {"score": self.W2(F.relu(self.W1(h))).squeeze(1)} # 2つの線形層を通過し、スカラースコアを返す

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.apply_edges(self.apply_edges)
            return g.edata["score"]

<div class="alert alert-info"><h4>Note</h4><p>
   ビルトイン関数は、速度とメモリの両方に最適化されているため、可能な限りビルトイン関数を使用することを推奨する。</p></div>

<div class="alert alert-info"><h4>Note</h4><p>
   :`message passing tutorial <3_message_passing>` を読んだことがある場合、
   ``apply_edges``の引数は、``update_all``のメッセージ関数とまったく同じ形式であることがわかるだろう。</p></div>
   </p></div>

## 学習ループ

ノード表現の計算とエッジスコアの計算を定義したら、モデル全体、損失関数、評価メトリックを定義する。


損失関数は、単純なバイナリ交差エントロピー損失である。

\begin{align}\mathcal{L} = -\sum_{u\sim v\in \mathcal{D}}\left( y_{u\sim v}\log(\hat{y}_{u\sim v}) + (1-y_{u\sim v})\log(1-\hat{y}_{u\sim v})) \right)\end{align}


評価指標は、このチュートリアルではAUCを用いる。


In [13]:
model = GraphSAGE(train_g.ndata["feat"].shape[1], 16)
# DotPredictor と MLPPredictor を置き換えることができる。
# pred = MLPPredictor(16)
pred = DotPredictor()


def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]
    )
    return F.binary_cross_entropy_with_logits(scores, labels)


def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]
    ).numpy()
    return roc_auc_score(labels, scores)

学習ループは以下の通りである:

<div class="alert alert-info"><h4>Note</h4><p>
   このチュートリアルにはバリデーションセットでの評価が含まれていません。実際には、バリデーションセットでのパフォーマンスに基づいて最良のモデルを保存して評価する必要がある。</p></div>




In [15]:
# ----------- 3. set up loss and optimizer -------------- #
# 今回は、損失は学習ループ内で計算される
optimizer = torch.optim.Adam(
    itertools.chain(model.parameters(), pred.parameters()), lr=0.01
)

# ----------- 4. training -------------------------------- #
all_logits = []
for e in range(100):
    # forward
    h = model(train_g, train_g.ndata["feat"])
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print("In epoch {}, loss: {}".format(e, loss))

# ----------- 5. check results ------------------------ #
from sklearn.metrics import roc_auc_score

with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print("AUC", compute_auc(pos_score, neg_score))


# Thumbnail credits: Link Prediction with Neo4j, Mark Needham
# sphinx_gallery_thumbnail_path = '_static/blitz_4_link_predict.png'

In epoch 0, loss: 0.25840190052986145
In epoch 5, loss: 0.25079312920570374
In epoch 10, loss: 0.24637673795223236
In epoch 15, loss: 0.23924537003040314
In epoch 20, loss: 0.2313515990972519
In epoch 25, loss: 0.2229684591293335
In epoch 30, loss: 0.2141314297914505
In epoch 35, loss: 0.20476028323173523
In epoch 40, loss: 0.19516490399837494
In epoch 45, loss: 0.18557053804397583
In epoch 50, loss: 0.17561843991279602
In epoch 55, loss: 0.16514258086681366
In epoch 60, loss: 0.1545877754688263
In epoch 65, loss: 0.14387385547161102
In epoch 70, loss: 0.1331481635570526
In epoch 75, loss: 0.12258487194776535
In epoch 80, loss: 0.11222700774669647
In epoch 85, loss: 0.10211115330457687
In epoch 90, loss: 0.09234435111284256
In epoch 95, loss: 0.08299131691455841
AUC 0.8109826823296871
